In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import time

from skimage.feature import hog
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label

from lesson_functions import *

In [2]:
# import os

# basedir = 'vehicles/'
# image_types = os.listdir(basedir)

# cars = []
# for imtype in image_types:
#         cars.extend(glob.glob(basedir+imtype+'/*'))

# print ('Number of Vehicle Images found:', len(cars))
# with open ("cars.txt", 'w') as f:
#     for fn in cars:
#         f.write(fn+'\n')

# basedir = 'non-vehicles/'
# image_types = os.listdir(basedir)

# notcars = []
# for imtype in image_types:
#         notcars.extend(glob.glob(basedir+imtype+'/*'))

# print ('Number of Non-Vehicle Images found:', len(notcars))
# with open ("notcars.txt", 'w') as f:
#     for fn in notcars:
#         f.write(fn+'\n')

In [3]:
dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]
color_space = dist_pickle["color_space"]
hog_channel = dist_pickle["hog_channel"]
spatial_feat = dist_pickle["spatial_feat"]
hist_feat = dist_pickle["hist_feat"]
hog_feat = dist_pickle["hog_feat"]

In [4]:
# import glob

# searchpath = './test_images/*'
# example_images = glob.glob(searchpath)

# images = []
# titles = [] 

# for img_src in example_images:
#     t1 = time.time()
#     img = mpimg.imread(img_src)
#     image = img.astype(np.float32)/255     # png images (scaled 0 to 1 by mpimg) -> jpg (scaled 0 to 255)
#     print(np.min(image), np.max(image))
    
#     mid = slide_window(image, x_start_stop=[None, None], y_start_stop=[400, 656], 
#                       xy_window=(96, 96), xy_overlap=(0.8, 0.8))
#     front = slide_window(image, x_start_stop=[None, None], y_start_stop=[496, 656], 
#                       xy_window=(160, 160), xy_overlap=(0.8, 0.8))
    
#     hot_windows1 = search_windows(image, mid, svc, X_scaler, color_space=color_space, 
#                         spatial_size=spatial_size, hist_bins=hist_bins, 
#                         orient=orient, pix_per_cell=pix_per_cell, 
#                         cell_per_block=cell_per_block, 
#                         hog_channel=hog_channel, spatial_feat=spatial_feat, 
#                         hist_feat=hist_feat, hog_feat=hog_feat) 
#     hot_windows2 = search_windows(image, front, svc, X_scaler, color_space=color_space, 
#                         spatial_size=spatial_size, hist_bins=hist_bins, 
#                         orient=orient, pix_per_cell=pix_per_cell, 
#                         cell_per_block=cell_per_block, 
#                         hog_channel=hog_channel, spatial_feat=spatial_feat, 
#                         hist_feat=hist_feat, hog_feat=hog_feat) 

#     # window_img = draw_boxes(img, hot_windows1, color=(0, 0, 255), thick=6)
#     # window_img = draw_boxes(window_img, hot_windows2, color=(0, 0, 255), thick=6)
#     heat_map = np.zeros_like(img[:,:,0])
#     heat_map = add_heat(heat_map, hot_windows1)
#     heat_map = add_heat(heat_map, hot_windows2)
#     # heat_map = apply_threshold(heat_map, 2)
#     labels = label(heat_map)
#     draw_img = draw_labeled_bboxes(np.copy(img), labels)
#     images.append(draw_img)
#     images.append(heat_map)
    
#     titles.append('')
#     titles.append('')
#     print(time.time()-t1, 'seconds to process one image, searching', (len(mid)+len(front)),'windows')

# fig = plt.figure(figsize=(15,25))
# visualize(fig, 6, 2, images, titles)
# plt.show()

In [5]:
# import glob

# searchpath = './test_images/*'
# example_images = glob.glob(searchpath)

# out_images = []
# out_titles = []



# for img_src in example_images:
#     img = mpimg.imread(img_src)
#     allheat = []
#     ystart = 400 
#     ystop = 528
#     scale = 1
#     out_img, heat_map = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    
    
#     #heat_map = apply_threshold(heat_map,1)
#     heat_map = np.clip(heat_map, 0, 255)
#     labels = label(heat_map)
#     draw_img = draw_labeled_bboxes(np.copy(img), labels)
#     out_images.append(draw_img)
#     out_images.append(heat_map)
#     out_titles.append(img_src[-12:])
#     out_titles.append(img_src[-12:])
#     # print(type(heatmap))
#     # print(heatmap)

# fig = plt.figure(figsize=(12,24))

# visualize(fig, 6, 2, out_images, out_titles)
# plt.show()

In [6]:
from collections import deque
history = deque(maxlen=20) 

def process_video(img):
    ystart = 400 
    ystop = 528
    scale = 1
    out_img, heat_map = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    # heat_map = apply_threshold(heat_map, 2)
    history.append(heat_map)
    ystart = 400 
    ystop = 656
    scale = 1.5
    out_img, heat_map = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    heat_map = apply_threshold(heat_map, 1)
    history.append(heat_map)
    if len(history) < 20:
        labels = label(heat_map)
    else:
        heatmap_sum = sum(history)
        heatmap_sum = np.clip(heatmap_sum, 0, 255)
        labels = label(heatmap_sum)
    # Draw bounding boxes
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return draw_img

In [7]:
# from collections import deque
# history = deque(maxlen=8) 

# def process_video(img):

#     image = img.astype(np.float32)/255

#     mid = slide_window(image, x_start_stop=[None, None], y_start_stop=[400, 656], 
#                       xy_window=(96, 96), xy_overlap=(0.8, 0.8))
#     front = slide_window(image, x_start_stop=[None, None], y_start_stop=[496, 656], 
#                       xy_window=(160, 160), xy_overlap=(0.8, 0.8))
    
#     hot_windows1 = search_windows(image, mid, svc, X_scaler, color_space=color_space, 
#                         spatial_size=spatial_size, hist_bins=hist_bins, 
#                         orient=orient, pix_per_cell=pix_per_cell, 
#                         cell_per_block=cell_per_block, 
#                         hog_channel=hog_channel, spatial_feat=spatial_feat, 
#                         hist_feat=hist_feat, hog_feat=hog_feat) 
#     hot_windows2 = search_windows(image, front, svc, X_scaler, color_space=color_space, 
#                         spatial_size=spatial_size, hist_bins=hist_bins, 
#                         orient=orient, pix_per_cell=pix_per_cell, 
#                         cell_per_block=cell_per_block, 
#                         hog_channel=hog_channel, spatial_feat=spatial_feat, 
#                         hist_feat=hist_feat, hog_feat=hog_feat) 
    
#     heatmap = np.zeros_like(img[:,:,0])
#     heat_map = add_heat(heatmap, hot_windows1)
#     heat_map = add_heat(heat_map, hot_windows2)
#     heat_map = apply_threshold(heat_map, 1)
#     history.append(heat_map)
#     if len(history) < 10:       
#         labels = label(heat_map)
#     else:
#         heatmap_sum = sum(history)
#         labels = label(heatmap_sum)
#     draw_img = draw_labeled_bboxes(np.copy(img), labels)

#     return draw_img

In [8]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [9]:
# test_output = 'test_video_output.mp4'
# clip = VideoFileClip("test_video.mp4")
# test_clip = clip.fl_image(process_video)
# test_clip.write_videofile(test_output, audio=False)

In [10]:
video_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
video_clip = clip1.fl_image(process_video)
video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [21:59<00:01,  1.04s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

